In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def call(f, *args, m=None):
    if m is None:
        s = "(" + ", ".join(map(repr, args)) + ")"
        m = "%s%s" % (f.__name__, s)
    print(m)
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv
        
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
        
def endbatch():
    call(h.endbatch)

def wait2(s):
    pass
wait = wait1

def agauto(sp):
    call(h.setag, 0, sp)
    
def setmg(sp):
    call(h.setmg, 1, sp)

def pid1():
    call(h.setconfig, "DO", "O2_P_Gain_(%/DO%)", 1.5)
    call(h.setconfig, "DO", 'O2_I_Time_(min)', 120)
    call(h.setconfig, "DO", 'O2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'O2_Beta', 0)
    
    
    call(h.setconfig, "DO", "N2_P_Gain_(%/DO%)", -5)
    call(h.setconfig, "DO", 'N2_I_Time_(min)', 50)
    call(h.setconfig, "DO", 'N2_D_Time_(min)', 0)
    call(h.setconfig, "DO", 'N2_Beta', 0)
    

In [ ]:
h = HelloApp('71.189.82.196:81')
wait = wait1
dooff()
pid1()
agauto(12)
setmg(0.5)

wait(3600*10)
startbatch("dopid test")
doauto(150)
setmg(0.2)
wait(3600*10)
endbatch()
startbatch("dopid test2")
doauto(100)
wait(3600*10)
endbatch()
startbatch("dopid test3")
doauto(50)
wait(3600*10)
endbatch()
dooff()

In [ ]:
h = HelloApp('71.189.82.196:81')
wait = wait1
dooff()
pid1()
agauto(12)
setmg(0.5)
startbatch("dopid test")
doauto(150)
setmg(0.2)
wait(3600*10)
endbatch()
startbatch("dopid test2")
doauto(100)
wait(3600*10)
endbatch()
startbatch("dopid test3")
doauto(50)
wait(3600*10)
endbatch()
dooff()

In [ ]:
h = HelloApp('71.189.82.196:81')
wait = wait1
dooff()
pid1()
agauto(12)
setmg(0.5)
wait(3600*10)
startbatch("dopid test4")
doauto(150)
setmg(0.2)
wait(3600*10)
endbatch()

# 3L Analysis

In [ ]:
h = HelloApp('71.189.82.196:81')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\DO\\Raw Data\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+"3L " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [ ]:
for s in (" (continued01)", 2,3):
    dl("dopid test%s"%s)

In [ ]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
    
def analyze(fn):
    c1 = cells.Find("DOPV(%)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"DOPV(%)")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    for v in "DON2FlowActualRequest(%)", "DOO2FlowControllerRequestLimited(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
        
def analf(f):
    open_file(f)
    analyze(f)

In [ ]:
files = [p+bn+"3L " + bt() + ".csv" for bn in ['dopid test%s'%d for d in (" (continued01)", 2,3)]]
xl = Excel()
for file in files: analf(file)

In [ ]:
fn = dl('dopid test4')
analf(fn)

In [ ]:
for c in ws.ChartObjects():
    a=c.Chart.Axes(1,1).MaximumScale = 10

In [ ]:
h.login()
bl = h.getbatches()
b = bl.names_to_batches['dopid test4']
d1 = b.start_time
d2 = b.stop_time
import datetime
d2 += datetime.timedelta(hours=6)
n = "dopid test4 extended"
fn = dl((n, d1, d2))
analf(fn)

### pH Stuff

In [3]:
def phoff():
    call(h.setph, 2, 0, 0, m="Set pH to Off Mode")
    
def phauto(sp):
    call(h.setph, 0, sp, m="Set pH to Auto Mode at %.2f"%sp)
    
def setconfig(group, mode, val):
    msg = "Set %s %s to %s" % (group, mode, val)
    call(h.setconfig, group, mode, val, m=msg)
    
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)
    
def phpid1():
    setconfig("pH", "CO2_P_Gain_(%/pH)", 15)
    setconfig("pH", 'CO2_I_Time_(min)', 80)
    setconfig("pH", 'CO2_D_Time_(min)', 0)
    setconfig("pH", 'CO2_Beta', 0)
    
    setconfig("pH", "Base_P_Gain_(%/pH)", -10)
    setconfig("pH", 'Base_I_Time_(min)', 80)
    setconfig("pH", 'Base_D_Time_(min)', 0)
    setconfig("pH", 'Base_Beta', 0)
    
def checkph(exp, margin):
    pv = call(h.gpmv)['ph']['pv']
    if not exp-margin <= pv <= exp+margin:
        print("PV CHECK FAILED")
    else:
        print("PV STABLE")

In [4]:
h = HelloApp('71.189.82.196:81')
wait = wait2
phoff()
phpid1()
agauto(12)
setmg(0.1)
phauto(7.5)
wait(3600*15)
checkph(7.5, 0.1)
nb("phpid", 0)
setmg(0.2)
phauto(7.3)
wait(3600*15)
endbatch()
nb("phpid", 0)
phauto(7.5)
wait(3600*15)
endbatch()

Set pH to Off Mode
Set pH CO2_P_Gain_(%/pH) to 15
Set pH CO2_I_Time_(min) to 80
Set pH CO2_D_Time_(min) to 0
Set pH CO2_Beta to 0
Set pH Base_P_Gain_(%/pH) to -10
Set pH Base_I_Time_(min) to 80
Set pH Base_D_Time_(min) to 0
Set pH Base_Beta to 0
setag(0, 12)
setmg(1, 0.1)
Set pH to Auto Mode at 7.50
getMainValues()
PV CHECK FAILED
getBatches()
BatchEntry: id: 19 name: phpid 0.4 user: user1 start time: 2017-05-24 17:03:13 stop time: 2017-05-24 17:03:14
BatchEntry: id: 17 name: phpid 0.3 user: user1 start time: 2017-05-24 17:03:11 stop time: 2017-05-24 17:03:12
BatchEntry: id: 15 name: phpid 0.2 user: user1 start time: 2017-05-24 16:30:25 stop time: 2017-05-24 16:30:26
BatchEntry: id: 13 name: phpid 0.1 user: user1 start time: 2017-05-24 16:30:24 stop time: 2017-05-24 16:30:25
BatchEntry: id: 11 name: test2809 1 user: user1 start time: 2017-05-18 11:21:30 stop time: 2017-05-24 12:58:06
BatchEntry: id: 9 name: test2809 user: user1 start time: 2017-05-18 10:22:27 stop time: 2017-05-18 11:1

KeyError: 'ID'